In [26]:
from arcgis.gis import GIS, Item
import os

item_id = 'b23955c522ec47548dc355bb5fd79d1c'
field_store_id_alias = 'Store ID'
file_root = 'demo_trips'

In [27]:
gis = GIS(url='https://commteamretail.maps.arcgis.com')
gis

In [28]:
feature_layer = Item(gis, item_id).layers[0]
feature_layer

<FeatureLayer url:"https://services.arcgis.com/PMTtzuTB6WiPuNSv/arcgis/rest/services/National_Store_Locations/FeatureServer/0">

In [29]:
field_store_id = [field.name for field in feature_layer.properties.fields if field.alias == field_store_id_alias][0]
field_store_id

'NationalStoreLocations_Local__3'

In [30]:
feature_set = feature_layer.query(out_fields=field_store_id, out_sr={'wkid': 4326})
feature_set

<FeatureSet> 3156 features

In [31]:
sdf = feature_set.df
sdf.drop('OBJECTID', axis=1, inplace=True)
sdf.columns = ['store_id', 'SHAPE']
sdf.head()

,store_id,SHAPE
0,96,"{'x': -86.49607497200131, 'y': 39.155960926931..."
1,97,"{'x': -122.18647465212695, 'y': 47.71139930792..."
2,98,"{'x': -122.10074581765247, 'y': 37.65401004768..."
3,99,"{'x': -76.83241134100487, 'y': 39.277317841688..."
4,100,"{'x': -119.32970923096121, 'y': 36.22602745380..."


In [32]:
from inrix import TAZ

In [33]:
taz = TAZ(
    app_id="c9fa4c15-b9ab-46d4-bcd9-a3eb33dd0ff8",
    hash_token="919745fda981ddccb3ffc4f07863480e624cfa3d"
)
taz

In [36]:
def get_exisiting(directory, file_root):
    f_list = [f for f in os.listdir(directory) if os.path.isfile(f) and f.endswith('.csv') and f.startswith(file_root)]
    f_list = [f.split('.')[0] for f in f_list]
    return [int(f.split('_').pop()) for f in f_list]

In [42]:
locnum_list = [store_id for store_id in sdf.store_id if store_id not in get_exisiting('.', file_root)]
locnum_list

[689,
 690,
 691,
 692,
 693,
 694,
 695,
 696,
 697,
 698,
 699,
 700,
 701,
 702,
 703,
 704,
 705,
 706,
 707,
 708,
 709,
 710,
 711,
 712,
 713,
 714,
 715,
 716,
 717,
 718,
 719,
 720,
 721,
 722,
 723,
 724,
 725,
 726,
 727,
 728,
 729,
 730,
 731,
 732,
 733,
 734,
 735,
 736,
 737,
 738,
 739,
 740,
 741,
 742,
 743,
 744,
 745,
 746,
 747,
 748,
 749,
 750,
 751,
 752,
 753,
 754,
 755,
 756,
 757,
 758,
 759,
 760,
 761,
 762,
 763,
 764,
 765,
 766,
 767,
 768,
 769,
 770,
 771,
 772,
 773,
 774,
 775,
 776,
 777,
 778,
 779,
 780,
 781,
 782,
 783,
 784,
 785,
 786,
 787,
 788,
 789,
 790,
 791,
 792,
 793,
 794,
 795,
 796,
 797,
 798,
 799,
 800,
 801,
 802,
 803,
 804,
 805,
 806,
 807,
 808,
 809,
 810,
 811,
 812,
 813,
 814,
 815,
 816,
 817,
 818,
 819,
 820,
 821,
 822,
 823,
 824,
 825,
 826,
 827,
 828,
 829,
 830,
 831,
 832,
 833,
 834,
 835,
 836,
 837,
 838,
 839,
 840,
 841,
 842,
 843,
 844,
 845,
 846,
 847,
 848,
 849,
 850,
 851,
 852,
 853,
 854,
 855

In [ ]:
for store_id in locnum_list:
    
    row = sdf.loc[sdf.store_id == store_id].iloc[0]
    
    sdf_this = taz.get_trip_destination_spatial_dataframe(row.SHAPE.y, row.SHAPE.x, '300m')
    sdf_this['store_id'] = row.store_id
    
    sdf_this.to_csv('demo_trips_{}.csv'.format(row.store_id))
    
    print('Trips sucessfully retrieved and saved for LocNum {}'.format(row.store_id))